## My little database JSON-LD import

#### Import relevant packages

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing
import bw_migrations

import os
import numpy as np
import pandas as pd

#### Create/set the working folder:

In [2]:
bd.projects.dir

PosixPath('/Users/mmendez/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682')

In [3]:
bd.projects.set_current('MLD_JSON_2')

#### Import JSON-LD:

In [4]:
path = '../7_BW2_Importing_USLCI/databases/My_little_database' # For some reason if I use the relative path it throws me an error
mld = bi.importers.JSONLDImporter(
    path, 
    "My_little_dataset_v0", 
    preferred_allocation="PHYSICAL_ALLOCATION"
)

#### Apply strategies to map JSON-LD to Brightway2 schema:

In [5]:
mld.apply_strategies()

Applying strategy: json_ld_allocate_datasets
Applying strategy: json_ld_get_normalized_exchange_locations
Applying strategy: json_ld_convert_unit_to_reference_unit
Applying strategy: json_ld_get_activities_list_from_rawdata
Applying strategy: json_ld_add_products_as_activities
Applying strategy: json_ld_get_normalized_exchange_units
Applying strategy: json_ld_add_activity_unit
Applying strategy: json_ld_rename_metadata_fields
Applying strategy: json_ld_location_name
Applying strategy: json_ld_remove_fields
Applying strategy: json_ld_fix_process_type
Applying strategy: json_ld_label_exchange_type
Applying strategy: json_ld_prepare_exchange_fields_for_linking
Applying strategy: add_database_name
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: normalize_units
Applied 17 strategies in 0.00 seconds

	Created 3 biosphere flows in separate database 'My_little_dataset_v0 biosphere'.
	Use either `.merge_biosphere_flows()` or `.write_separ

#### Check database dictionaries:

In [6]:
bd.databases # As expected, nothing was written.

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

#### Write the biosphere database:

In [7]:
mld.write_separate_biosphere_database()

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/20/2022 17:23:01
  Finished: 09/20/2022 17:23:01
  Total time elapsed: 00:00:00
  CPU %: 2.50
  Memory %: 0.49
Created database: My_little_dataset_v0 biosphere


In [8]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

#### Write the technosphere database:

In [9]:
bio = bd.Database('My_little_dataset_v0 biosphere')

In [10]:
mld.write_database()

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/20/2022 17:23:01
  Finished: 09/20/2022 17:23:01
  Total time elapsed: 00:00:00
  CPU %: 2.30
  Memory %: 0.50
Created database: My_little_dataset_v0


Brightway2 SQLiteBackend: My_little_dataset_v0

<div class="alert alert-block alert-warning">
<b>To fix:</b>
    <li>The importer does not recognize the geolocations.</li>
    <li>The importer does not import the LCIA methods. <i>Hint:</i> Check bw2setup() function, this one loads LCIAs and biosphere flows.</li>
</div>

In [11]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

In [12]:
db = bd.Database('My_little_dataset_v0')

#### Let's look at the biosphere flows:

In [13]:
[act for act in bio]

['Bad stuff' (, None, ('Elementary flows', 'emissions')),
 'Crude oil' (, None, ('Elementary flows', 'resources')),
 'Virgin metals' (, None, ('Elementary flows', 'resources'))]

#### And now let's look at the our processes and products:

In [14]:
[(act.as_dict(), act['unit']) for act in bio]

[({'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
   'name': 'Bad stuff',
   'categories': ('Elementary flows', 'emissions'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 21},
  ''),
 ({'code': '8849be54-1b13-4d7e-85f6-2297817333f2',
   'name': 'Crude oil',
   'categories': ('Elementary flows', 'resources'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 20},
  ''),
 ({'code': '7528830a-7344-43be-b484-ac4dc625f272',
   'name': 'Virgin metals',
   'categories': ('Elementary flows', 'resources'),
   'CAS number': None,
   'database': 'My_little_dataset_v0 biosphere',
   'unit': '',
   'type': 'emission',
   'id': 19},
  '')]

In [15]:
[(act, act['type'], act['unit']) for act in db]

[('Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
  'product',
  ''),
 ('Plastic manufacturing' (kilogram, Djibouti, None), 'process', 'kilogram'),
 ('Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')), 'product', ''),
 ('Stainless steel manufacturing' (kilogram, Djibouti, None),
  'process',
  'kilogram'),
 ('Bottle assembly' (number_of_items, Djibouti, None),
  'process',
  'number_of_items'),
 ('Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
  'product',
  '')]

In [16]:
[(act['name'],[exc for exc in act.exchanges()]) for act in db if act['type'] == 'process']

[('Bottle assembly',
  [Exchange: 1.0  'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.25  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.1  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.3  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>]),
 ('Stainless steel manufacturing',
  [Exchange: 1.0  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Virgin metals' (, None, ('Elementary flows', 'resources')) to 'Stainless steel manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 3.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Stainless steel

In [17]:
[(act['name'],[exc for exc in act.exchanges()]) for act in db if act['type'] == 'process']

[('Plastic manufacturing',
  [Exchange: 1.0  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 2.0  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>,
   Exchange: 1.0  'Crude oil' (, None, ('Elementary flows', 'resources')) to 'Plastic manufacturing' (kilogram, Djibouti, None)>]),
 ('Bottle assembly',
  [Exchange: 1.0  'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.25  'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.1  'Bad stuff' (, None, ('Elementary flows', 'emissions')) to 'Bottle assembly' (number_of_items, Djibouti, None)>,
   Exchange: 0.3  'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')) to 'Bottle assembly' (number_of_items, Djibouti, None)>]

#### Which ones are processes and which ones are products? Let's also get their codes:

In [18]:
[(act['name'], act['type'], act['code']) for act in db]

[('Plastic', 'product', '62714200-1a0d-43fb-9b48-99df3f233c94'),
 ('Stainless steel manufacturing',
  'process',
  '7e5ec332-09fd-4706-8373-3f140a539028'),
 ('Bottle assembly', 'process', 'fae6799b-7326-452c-92b3-76758bbcac22'),
 ('Stainless steel', 'product', '3355a1b9-8fbf-40de-b449-ea6399a8a323'),
 ('Plastic manufacturing', 'process', '2fc6deea-6437-4b9f-bf91-89bca44d30f0'),
 ('Bottle', 'product', 'b806c2cd-d563-43c4-a0c9-9c7dd5d513d3')]

### Let's manually add a LCIA method
This importer does not recognize the LCIA methods, therefore we need to add them manually. I will debug this later!

First, let's find the `Bad stuff` biosphere flow.

In [19]:
bad_flow = [act for act in bio if act['name'] == 'Bad stuff'][0]
bad_flow.as_dict()

{'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
 'name': 'Bad stuff',
 'categories': ('Elementary flows', 'emissions'),
 'CAS number': None,
 'database': 'My_little_dataset_v0 biosphere',
 'unit': '',
 'type': 'emission',
 'id': 21}

In [20]:
myLCIAdata = [[(bad_flow['database'], bad_flow['code']), 2.0]] # A method list needs: a reference to the flow: tuple (database, 'code')), a characterization factor number, and localization (if no localization is given, 'GLO' is used)
method_key = ('MacIM', 'Global warming', 'total')
my_method = bd.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register()
my_method.write(myLCIAdata)

#### Now we define a functional unit:
This one might be a bit counterintuitive, our functional unit here is **Impact of assembling 5 bottles**, intuintively one would select the activity, but bw2 selects the flow coming out of the `Bottle assembly` activity (i.e. `Bottle`, which is a `product` not a `process`).

In [21]:
activity = [act for act in db if act['name'] == 'Bottle'][0]

In [22]:
functional_unit = {activity : 5} #Impact of 5 bottles

#### Run the LCA!

In [23]:
lca = bc.LCA(functional_unit, method_key) 

In [24]:
lca.lci()   # Builds matrices, solves the system, generates an LCI matrix.

In [27]:
lca2 = bc.LCA(functional_unit, method_key) 

In [29]:
len(dir(lca2))

72

In [30]:
len(dir(lca))

79

In [31]:
lca.lcia()  # Characterization, i.e. the multiplication of the elements  
            # of the LCI matrix with characterization factors from the chosen method


In [32]:
lca.score    # Returns the score, i.e. the sum of the characterized inventory

15.00000037252903

In [35]:
lca2.lcia()

In [38]:
dir(lca2)

['_LCA__check_demand',
 '__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_switch',
 'biosphere_matrix',
 'biosphere_mm',
 'build_demand_array',
 'characterization_matrix',
 'characterization_mm',
 'characterized_inventory',
 'decompose_technosphere',
 'demand',
 'demand_array',
 'dicts',
 'ensure_bw2data_available',
 'has',
 'inventory',
 'invert_technosphere_matrix',
 'keep_first_iteration',
 'lci',
 'lci_calculation',
 'lcia',
 'lcia_calculation',
 'load_lci_data',
 'load_lcia_data',
 'load_normalization_data',
 'load_weighting_data',
 'matrix_labels',
 'method',
 'norm

Scoooooooore!!!